In [1]:
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Subset, ConcatDataset

import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
import random
import json

In [2]:
import segmentation_models_pytorch as smp
from segmentation_models_pytorch.losses import DiceLoss
from segmentation_models_pytorch.utils.metrics import IoU, Fscore, Accuracy

In [3]:
import matplotlib.pyplot as plt

# helper function for data visualization
def visualize(**images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(16, 5))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image, 'gray')
    plt.show()

In [4]:
import random

SEED = 0
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

print(np.random.rand(5), torch.randn(5))

[0.5488135  0.71518937 0.60276338 0.54488318 0.4236548 ] tensor([ 1.5410, -0.2934, -2.1788,  0.5684, -1.0845])


# Settings

In [5]:
root = os.getcwd()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

BATCH = 4

GEN_IN_CHANNELS = 1
GEN_N_CLASSES = 1

DIS_IN_CHANNELS = 1
DIS_N_CLASSES = 2

E_IN_CHANNELS = 1
E_N_CLASSES = 2

EPOCH = 200

### Model Settings

#### encoder

In [6]:
ENCODER = "resnet152"
ENCODER_WEIGHT = None

#### decoder

In [7]:
DECODER_ATT = "scse"

#### head

In [8]:
GEN_ACT = "sigmoid"
DIS_ACT = "softmax"
E_ACT = "softmax"

# Epochs

In [9]:
from codes.losses import SSIMLoss
from codes.losses import MAELoss
from pytorch_msssim import ssim
from codes.metrics import PSNR, SNR, ContourEval
from codes.activation import Activation
from codes.utils import hu_clip_tensor
from codes.losses import PerceptualLoss
from kornia.filters.sobel import Sobel

In [10]:
def set_requires_grad(nets, requires_grad=False):
    """Set requies_grad=Fasle for all the networks to avoid unnecessary computations
    Parameters:
    nets (network list)   -- a list of networks
    requires_grad (bool)  -- whether the networks require gradients or not
    """
    if not isinstance(nets, list):
        nets = [nets]
        for net in nets:
            if net is not None:
                for param in net.parameters():
                    param.requires_grad = requires_grad

In [11]:
def replace_relu_to_leakyReLU(model):
    for child_name, child in model.named_children():
        if isinstance(child, nn.ReLU):
            setattr(model, child_name, nn.LeakyReLU())
        else:
            replace_relu_to_leakyReLU(child)

In [12]:
def replace_bn_to_instanceNorm(model):
    for child_name, child in model.named_children():
        if isinstance(child, nn.BatchNorm2d):
            bn = getattr(model, child_name)
            setattr(model, child_name, nn.InstanceNorm2d(bn.num_features))
        else:
            replace_bn_to_instanceNorm(child)

In [13]:
perceptual_ext = PerceptualLoss()

In [14]:
sobel_filter = Sobel().to(device)

In [15]:
@torch.no_grad()
def eval_epoch(generator, tf_discriminator, edge_discriminator, dataloader, device):
  
    # change mode to train and move to current device
    generator = generator.eval().to(device)
    tf_discriminator = tf_discriminator.eval().to(device)
    edge_discriminator = edge_discriminator.eval().to(device)
    
    ssim_ = []
    psnr_ = []
    mae_ = []
    snr_ = []
    
    air_ = []
    bone_ = []
    cont_ = []
 
    tf_acc = []
    edge_acc = []
    
    for index, data in tqdm(enumerate(dataloader)):

        x, y, air_x, bone_x, *_ = data
        
        B, T, C, H, W = x.size()
        x = x.to(device).view(-1, C, H, W)
        y = y.to(device).view(-1, 1, H, W)
        air_x = air_x.to(device).view(-1, 1, H, W)
        bone_x = bone_x.to(device).view(-1, 1, H, W)

        y_pr = generator(x.float())
        y_pr = Activation(name=GEN_ACT)(y_pr) # zipped value to [0, 1]
    
       ############################
        # (1) Eval True/Fake D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################        
        y_cls = tf_discriminator(y.float())
        for _y_cls in y_cls:
            _y_cls = _y_cls.view(B, T, 2)[:, 1, :]
            _B, _C = _y_cls.size()
            total = _B
            _y_cls = Activation(name=DIS_ACT)(_y_cls)
            _, _y_cls = torch.max(_y_cls.data, 1)
            correct = (_y_cls == torch.zeros(_B, dtype=torch.long, device=device)).sum().item()
            tf_acc += [correct / total]        
        
        y_pr_cls = tf_discriminator(y_pr)
        for _y_pr_cls in y_pr_cls:
            _y_pr_cls = _y_pr_cls.view(B, T, 2)[:, 1, :]
            _B, _C = _y_pr_cls.size()
            total = _B
            _y_pr_cls = Activation(name=DIS_ACT)(_y_pr_cls)
            _, _y_pr_cls = torch.max(_y_pr_cls.data, 1)
            correct = (_y_pr_cls == torch.ones(_B, dtype=torch.long, device=device)).sum().item()
            tf_acc += [correct / total]

 
       ############################
        # (1.1) Eval Edge D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################  

        edge_y_pr = sobel_filter(y_pr)
        edge_y = sobel_filter(y)
        
        edge_y_cls = edge_discriminator(edge_y.float())
        for _edge_y_cls in edge_y_cls:
            _edge_y_cls = _edge_y_cls.view(B, T, 2)[:, 1, :]
            _B, _C = _edge_y_cls.size()
            total = _B
            _edge_y_cls = Activation(name=DIS_ACT)(_edge_y_cls)
            _, _edge_y_cls = torch.max(_edge_y_cls.data, 1)
            correct = (_edge_y_cls == torch.zeros((_B), dtype=torch.long, device=device)).sum().item()
            edge_acc += [correct / total]          
        
        edge_pr_cls = edge_discriminator(edge_y_pr)
        for _edge_pr_cls in edge_pr_cls:
            _edge_pr_cls = _edge_pr_cls.view(B, T, 2)[:, 1, :]
            _B, _C = _edge_pr_cls.size()
            total = _B
            _edge_pr_cls = Activation(name=DIS_ACT)(_edge_pr_cls)
            _, _edge_pr_cls = torch.max(_edge_pr_cls.data, 1)
            correct = (_edge_pr_cls == torch.ones((_B), dtype=torch.long, device=device)).sum().item()
            edge_acc += [correct / total]        

        ############################
        # (2) Eval G network: maximize log(D(G(z)))
        ###########################
        x = x.view(B, T, C, H, W)[:, 1, :]
        y = y.view(B, T, 1, H, W)[:, 1, :]
        y_pr = y_pr.view(B, T, 1, H, W)[:, 1, :]
        air_x = air_x.view(B, T, 1, H, W)[:, 1, :]
        bone_x = bone_x.view(B, T, 1, H, W)[:, 1, :]
        
        # main
        _ssim = ssim(y.float(), y_pr, data_range=1.0, size_average=True) # return (N,)
        _psnr = PSNR()(y_pr, y.float(), 1.0)
        _snr = SNR()(y_pr, y.float())
        _mae = MAELoss()(y_pr.float(), y.float())

        ssim_ += [_ssim.item()]
        psnr_ += [_psnr.item()]
        mae_ += [_mae.item()]
        snr_ += [_snr.item()]
        
        # auxilary
        _min = VIEW_BOUND[0]
        _max = VIEW_BOUND[1]
        air_window = AIR_BOUND
        upper = ((air_window[1]) - (_min))/(_max-(_min))
        lower = ((air_window[0]) - (_min))/(_max-(_min))
        air_pr = hu_clip_tensor(y_pr.double(), (lower, upper), None, True)
        dice = Fscore()(air_pr, air_x)
        air_ += [dice.item()]
        
        bone_window = BONE_BOUND
        upper = ((bone_window[1]) - (_min))/(_max-(_min))
        lower = ((bone_window[0]) - (_min))/(_max-(_min))
        bone_pr = hu_clip_tensor(y_pr.double(), (lower, upper), None, True)
        dice = Fscore()(bone_pr, bone_x)
        bone_ += [dice.item()]
        
        for b in range(x.size()[0]):
            cont = ContourEval()(y_pr[b, :, :, :], x[b, :, :, :])
            cont_ += [cont.item()]

    return  sum(ssim_)/len(ssim_), sum(psnr_)/len(psnr_), sum(snr_)/len(snr_), sum(mae_)/len(mae_), \
                    sum(air_)/len(air_), sum(bone_)/len(bone_), sum(cont_)/len(cont_), sum(tf_acc)/len(tf_acc), sum(edge_acc)/len(edge_acc)

In [16]:
@torch.no_grad()
def test_epoch(iid, model, dataloader, device, save=False, path=None):
  
    # change mode to train and move to current device
    model = model.eval().to(device)
    
    iid = iid
    for index, data in tqdm(enumerate(dataloader)):

        x, y, air_x, bone_x, *_ = data
        
        B, T, C, H, W = x.size()
        x = x.to(device).view(-1, C, H, W)
        y = y.to(device).view(-1, 1, H, W)
        air_x = air_x.to(device).view(-1, 1, H, W)
        bone_x = bone_x.to(device).view(-1, 1, H, W)

        y_pr = model(x.float())
        y_pr = Activation(name=GEN_ACT)(y_pr) # zipped value to [0, 1]
 
        x = x.view(B, T, C, H, W)[:, 1, :]
        y = y.view(B, T, 1, H, W)[:, 1, :]
        air_x = air_x.view(B, T, 1, H, W)[:, 1, :]
        bone_x = bone_x.view(B, T, 1, H, W)[:, 1, :]
        y_pr = y_pr.view(B, T, 1, H, W)[:, 1, :]

        edge_x = sobel_filter(x)
        edge_y_pr = sobel_filter(y_pr)
        edge_y = sobel_filter(y)

        _min = VIEW_BOUND[0]
        _max = VIEW_BOUND[1]
        air_window = AIR_BOUND
        upper = ((air_window[1]) - (_min))/(_max-(_min))
        lower = ((air_window[0]) - (_min))/(_max-(_min))
        air_pr = hu_clip_tensor(y_pr.double(), (lower, upper), None, True)
        
        bone_window = BONE_BOUND
        upper = ((bone_window[1]) - (_min))/(_max-(_min))
        lower = ((bone_window[0]) - (_min))/(_max-(_min))
        bone_pr = hu_clip_tensor(y_pr.double(), (lower, upper), None, True)

        for b in range(x.shape[0]):           
            tmp_y = y[b, :, :, :].unsqueeze(0).float()
            tmp_y_pr = y_pr[b, :, :, :].unsqueeze(0)
            tmp_air = air_x[b, :, :, :].unsqueeze(0)
            tmp_air_pr = air_pr[b, :, :, :].unsqueeze(0)
            tmp_bone = bone_x[b, :, :, :].unsqueeze(0)
            tmp_bone_pr = bone_pr[b, :, :, :].unsqueeze(0)
            
            _ssim = ssim(tmp_y, tmp_y_pr, data_range=1.0, size_average=True) # return (N,)
            _psnr = PSNR()(tmp_y_pr, tmp_y, 1.0)
            _snr = SNR()(tmp_y_pr, tmp_y)
            _mae = MAELoss()(tmp_y_pr.float(), tmp_y.float())
            _air = Fscore()(tmp_air_pr, tmp_air)
            _bone = Fscore()(tmp_bone_pr, tmp_bone)
            _cont = ContourEval()(tmp_y_pr, x[b, :, :, :])
            
            __cbct = (x.squeeze().cpu().numpy() * 255).astype(np.uint8)
            __ct = (tmp_y.squeeze().cpu().numpy() * 255).astype(np.uint8)
            __ct_pred = (tmp_y_pr.squeeze().cpu().numpy() * 255).astype(np.uint8)
            __ct_masked = __ct_pred * tmp_air.squeeze().cpu().numpy()
            __air = (tmp_air.squeeze().cpu().numpy() * 255).astype(np.uint8)
            __air_pr = (tmp_air_pr.squeeze().cpu().numpy() * 255).astype(np.uint8)
            __bone = (tmp_bone.squeeze().cpu().numpy() * 255).astype(np.uint8)
            __bone_pr = (tmp_bone_pr.squeeze().cpu().numpy() * 255).astype(np.uint8)
            __edge_pr = (edge_y_pr[b, :].squeeze().cpu().numpy() * 255).astype(np.uint8)
            __edge_x = (edge_x.squeeze().cpu().numpy() * 255).astype(np.uint8)
            __edge_y = (edge_y.squeeze().cpu().numpy() * 255).astype(np.uint8)

            
            if save:
                path_dir = os.path.join(path, "file_{}".format(iid))
                try:
                    os.mkdir(path_dir)
                except FileExistsError:
                    pass
                
                cv2.imwrite(os.path.join(path_dir, "cbct.jpg"), __cbct)
                cv2.imwrite(os.path.join(path_dir, "ct.jpg"), __ct)
                cv2.imwrite(os.path.join(path_dir, "ct_pred.jpg"), __ct_pred)
                cv2.imwrite(os.path.join(path_dir, "air.jpg"), __air)
                cv2.imwrite(os.path.join(path_dir, "air_pred.jpg"), __air_pr)
                cv2.imwrite(os.path.join(path_dir, "bone.jpg"), __bone)
                cv2.imwrite(os.path.join(path_dir, "bone_pred.jpg"), __bone_pr)
                cv2.imwrite(os.path.join(path_dir, "edge_pred.jpg"), __edge_pr)
                cv2.imwrite(os.path.join(path_dir, "edge_cbct.jpg"), __edge_x)
                cv2.imwrite(os.path.join(path_dir, "edge_ct.jpg"), __edge_y)
                
            iid += 1

# Discriminator

In [17]:
from codes.activation import Activation
import torchvision.models as models
from codes.losses import MultiScaleHeads

In [18]:
class Discriminator(nn.Module):
    def __init__(self, in_channel=1, n_classes=2, activation=None):
        
        super(Discriminator, self).__init__()
        
        encoder = smp.Unet(encoder_name="resnet18", in_channels=in_channel, classes=n_classes).encoder
        self.encoder = encoder
        self.fc = MultiScaleHeads(n_classes=n_classes, channels=(512, ), activation=None)
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.fc([x[-1]])
        return x

# Generator

In [19]:
import segmentation_models_pytorch as smp
from codes.decoder import UnetDecoder
from segmentation_models_pytorch.base.heads import SegmentationHead

In [20]:
class Generator(nn.Module):
    def __init__(self, encoder_name, encoder_weights, in_channels, classes, attention_type):
        
        super(Generator, self).__init__()
        
        self.encoder = smp.Unet(encoder_name=encoder_name, encoder_weights=encoder_weights, in_channels=in_channels, classes=classes).encoder
        self.decoder = UnetDecoder(
                            encoder_channels=[in_channels, 0, 0, 0, 0, 2048],
                            decoder_channels=[512, 256, 128, 64, 16],
                            n_blocks=5,
                            use_batchnorm=True,
                            attention_type=attention_type)
        self.head =  SegmentationHead(
                        in_channels=16,
                        out_channels=classes,
                        activation=None)
        
        
    def forward(self, x):
        x = self.encoder(x)[-1]
        x = self.decoder(*(None, None, None, None, x))
        x = self.head(x)
        
        return x

# Read Data

In [21]:
import glob
from codes.dataset import DicomSegmentDataset, DicomsSegmentDataset
import codes.augmentation as aug
from codes.temporal_shift import make_temporal_shift

In [22]:
# run_name = wandb.run.name
run_name = "apricot-oath-471"
ELECTRON = False
G_COORD = False
L_COORD= False

N_SEGMENT = 3

In [23]:
VIEW_BOUND = (-500, 500)
AIR_BOUND = (-500, -499)
BONE_BOUND = (255, 256)
if ELECTRON:
    VIEW_BOUND = (0.5, 1.5)
    AIR_BOUND = (0.5, 0.5009)
    BONE_BOUND = (1.2, 1.2009)  

In [24]:
checkpoint = torch.load(os.path.join("weight-gan", "{}.pth".format(run_name)))

In [25]:
model = smp.Unet(encoder_name=ENCODER, 
                  encoder_weights=ENCODER_WEIGHT, 
                  in_channels=GEN_IN_CHANNELS, 
                  classes=GEN_N_CLASSES,
                 decoder_attention_type=DECODER_ATT)
replace_relu_to_leakyReLU(model)
replace_bn_to_instanceNorm(model)
make_temporal_shift(model, N_SEGMENT, n_div=4, place="blockres")

model.load_state_dict(checkpoint["model"])

=> n_segment per stage: [3, 3, 3, 3]
=> Using n_round 2 to insert temporal shift
=> Processing stage with 3 blocks residual
=> Using fold div: 4
=> Using fold div: 4
=> Processing stage with 8 blocks residual
=> Using fold div: 4
=> Using fold div: 4
=> Using fold div: 4
=> Using fold div: 4
=> Processing stage with 36 blocks residual
=> Using fold div: 4
=> Using fold div: 4
=> Using fold div: 4
=> Using fold div: 4
=> Using fold div: 4
=> Using fold div: 4
=> Using fold div: 4
=> Using fold div: 4
=> Using fold div: 4
=> Using fold div: 4
=> Using fold div: 4
=> Using fold div: 4
=> Using fold div: 4
=> Using fold div: 4
=> Using fold div: 4
=> Using fold div: 4
=> Using fold div: 4
=> Using fold div: 4
=> Processing stage with 3 blocks residual
=> Using fold div: 4
=> Using fold div: 4


<All keys matched successfully>

In [26]:
tf_discriminator = Discriminator(in_channel=DIS_IN_CHANNELS, n_classes=DIS_N_CLASSES, activation=DIS_ACT)
replace_relu_to_leakyReLU(tf_discriminator)
replace_bn_to_instanceNorm(tf_discriminator)
make_temporal_shift(tf_discriminator, N_SEGMENT, n_div=4, place="blockres")

tf_discriminator.load_state_dict(checkpoint["tf_discriminator"])

=> n_segment per stage: [3, 3, 3, 3]
=> Processing stage with 2 blocks residual
=> Using fold div: 4
=> Using fold div: 4
=> Processing stage with 2 blocks residual
=> Using fold div: 4
=> Using fold div: 4
=> Processing stage with 2 blocks residual
=> Using fold div: 4
=> Using fold div: 4
=> Processing stage with 2 blocks residual
=> Using fold div: 4
=> Using fold div: 4


<All keys matched successfully>

In [27]:
edge_discriminator = Discriminator(in_channel=E_IN_CHANNELS, n_classes=E_N_CLASSES, activation=E_ACT)
replace_relu_to_leakyReLU(edge_discriminator)
replace_bn_to_instanceNorm(edge_discriminator)
make_temporal_shift(edge_discriminator, N_SEGMENT, n_div=4, place="blockres")

edge_discriminator.load_state_dict(checkpoint["edge_discriminator"])

=> n_segment per stage: [3, 3, 3, 3]
=> Processing stage with 2 blocks residual
=> Using fold div: 4
=> Using fold div: 4
=> Processing stage with 2 blocks residual
=> Using fold div: 4
=> Using fold div: 4
=> Processing stage with 2 blocks residual
=> Using fold div: 4
=> Using fold div: 4
=> Processing stage with 2 blocks residual
=> Using fold div: 4
=> Using fold div: 4


<All keys matched successfully>

# Pelvic

## Pelvic Test CBCT

In [28]:
test_case_path = 'raw/test/*_*'
paths = sorted(glob.glob(test_case_path))

In [29]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [30]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "test_pelvic_cbct"))
except FileExistsError:
    pass

In [31]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomSegmentDataset(cbct_path=paths[i+1], ct_path=paths[i],
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=False, electron=ELECTRON, position="pelvic", segment=N_SEGMENT, g_coord=G_COORD, l_coord=L_COORD)
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "test_pelvic_cbct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "test_pelvic_cbct", patient_id)) 
    iid += len(scans)

28it [00:05,  5.46it/s]
27it [00:04,  6.66it/s]
26it [00:03,  6.61it/s]
28it [00:04,  6.69it/s]
27it [00:04,  6.65it/s]
28it [00:04,  6.57it/s]
27it [00:04,  6.56it/s]


In [32]:
testset = DicomsSegmentDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=False, electron=ELECTRON, position="pelvic", segment=N_SEGMENT, g_coord=G_COORD, l_coord=L_COORD)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

191it [00:14, 12.81it/s]


(0.8138012605187781,
 22.6786757364323,
 13.682947723029171,
 0.031092364828624028,
 0.9962027991963913,
 0.8493048192755555,
 0.3612836842761614,
 0.5,
 0.5)

## Pelvic Test CT

In [33]:
test_case_path = 'raw/test/*_*'
paths = sorted(glob.glob(test_case_path))

In [34]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [35]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "test_pelvic_ct"))
except FileExistsError:
    pass

In [36]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomSegmentDataset(cbct_path=paths[i+1], ct_path=paths[i], 
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=True, electron=ELECTRON, position="pelvic", segment=N_SEGMENT, g_coord=G_COORD, l_coord=L_COORD)
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "test_pelvic_ct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "test_pelvic_ct", patient_id)) 
    iid += len(scans)

28it [00:04,  6.55it/s]
27it [00:04,  6.56it/s]
26it [00:03,  6.50it/s]
28it [00:04,  6.57it/s]
27it [00:04,  6.52it/s]
28it [00:04,  6.48it/s]
27it [00:04,  6.31it/s]


In [37]:
testset = DicomsSegmentDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=True, electron=ELECTRON, position="pelvic", segment=N_SEGMENT, g_coord=G_COORD, l_coord=L_COORD)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

191it [00:14, 12.76it/s]


(0.900890488899191,
 26.41364314293986,
 17.417915124543676,
 0.021819801411118493,
 0.9965161510260492,
 0.8263424171258713,
 0.39166875386425337,
 0.5,
 0.5)

## Pelvic L1 CBCT

In [38]:
test_case_path = 'L1_pelvic_processed/test/*_*'
paths = sorted(glob.glob(test_case_path))

In [39]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [40]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "L1_pelvic_cbct"))
except FileExistsError:
    pass

In [41]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomSegmentDataset(cbct_path=paths[i+1], ct_path=paths[i], 
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=False, electron=ELECTRON, position="pelvic", segment=N_SEGMENT, g_coord=G_COORD, l_coord=L_COORD)
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "L1_pelvic_cbct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "L1_pelvic_cbct", patient_id)) 
    iid += len(scans)

26it [00:04,  6.41it/s]
30it [00:04,  6.45it/s]
26it [00:04,  6.34it/s]
32it [00:05,  6.03it/s]
36it [00:09,  3.63it/s]
28it [00:07,  3.62it/s]
27it [00:07,  3.67it/s]
29it [00:07,  3.64it/s]
26it [00:07,  3.60it/s]
27it [00:07,  3.59it/s]
27it [00:07,  3.61it/s]


In [42]:
testset = DicomsSegmentDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=False, electron=ELECTRON, position="pelvic", segment=N_SEGMENT, g_coord=G_COORD, l_coord=L_COORD)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

314it [00:47,  6.61it/s]


(0.8139042379749808,
 22.63660915034592,
 13.07773541796739,
 0.0319929887202515,
 0.9949758114482931,
 0.8426183988395263,
 0.3914063014327341,
 0.5015923566878981,
 0.5031847133757962)

## Pelvic L1 CT

In [43]:
test_case_path = 'L1_pelvic_processed/test/*_*'
paths = sorted(glob.glob(test_case_path))

In [44]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [45]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "L1_pelvic_ct"))
except FileExistsError:
    pass

In [46]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomSegmentDataset(cbct_path=paths[i+1], ct_path=paths[i], 
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=True, electron=ELECTRON, position="pelvic", segment=N_SEGMENT, g_coord=G_COORD, l_coord=L_COORD)
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "L1_pelvic_ct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "L1_pelvic_ct", patient_id)) 
    iid += len(scans)

26it [00:07,  3.53it/s]
30it [00:08,  3.55it/s]
26it [00:07,  3.50it/s]
32it [00:08,  3.62it/s]
36it [00:10,  3.55it/s]
28it [00:08,  3.50it/s]
27it [00:07,  3.52it/s]
29it [00:08,  3.56it/s]
26it [00:07,  3.52it/s]
27it [00:07,  3.49it/s]
27it [00:07,  3.50it/s]


In [47]:
testset = DicomsSegmentDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=True, electron=ELECTRON, position="pelvic", segment=N_SEGMENT, g_coord=G_COORD, l_coord=L_COORD)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

314it [00:48,  6.51it/s]


(0.9059355843598675,
 26.320614504966006,
 16.761740818145167,
 0.023338691458983974,
 0.9971219880504772,
 0.8428082253478475,
 0.4671847665575659,
 0.5,
 0.5)

## Pelvic L2 CBCT

In [48]:
test_case_path = 'L2_pelvic_processed/test/*_*'
paths = sorted(glob.glob(test_case_path))

In [49]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [50]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "L2_pelvic_test"))
except FileExistsError:
    pass

In [51]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomSegmentDataset(cbct_path=paths[i+1], ct_path=paths[i], 
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=False, electron=ELECTRON, position="pelvic", segment=N_SEGMENT, g_coord=G_COORD, l_coord=L_COORD)
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "L2_pelvic_test", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "L2_pelvic_test", patient_id)) 
    iid += len(scans)

31it [00:09,  3.35it/s]
27it [00:07,  3.60it/s]
28it [00:07,  3.62it/s]
27it [00:07,  3.64it/s]
27it [00:07,  3.58it/s]
26it [00:07,  3.63it/s]
30it [00:08,  3.63it/s]
27it [00:07,  3.61it/s]
29it [00:08,  3.61it/s]
26it [00:07,  3.59it/s]
26it [00:07,  3.58it/s]
27it [00:07,  3.57it/s]
26it [00:07,  3.58it/s]
27it [00:07,  3.53it/s]
26it [00:07,  3.57it/s]
30it [00:08,  3.60it/s]
26it [00:07,  3.57it/s]
26it [00:07,  3.60it/s]
26it [00:07,  3.59it/s]
27it [00:07,  3.62it/s]
27it [00:07,  3.62it/s]
26it [00:07,  3.59it/s]
29it [00:08,  3.58it/s]
29it [00:08,  3.62it/s]


In [52]:
testset = DicomsSegmentDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=False, electron=ELECTRON, position="pelvic", segment=N_SEGMENT, g_coord=G_COORD, l_coord=L_COORD)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

656it [01:48,  6.06it/s]


(0.7878929187611836,
 21.432760575922526,
 11.88740349433771,
 0.03545993384507644,
 0.9963997850806926,
 0.8581771867947948,
 0.429065261944765,
 0.4961890243902439,
 0.5007621951219512)

## Pelvic L2 CT

In [53]:
test_case_path = 'L2_pelvic_processed/test/*_*'
paths = sorted(glob.glob(test_case_path))

In [54]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [55]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "L2_pelvic_ct"))
except FileExistsError:
    pass

In [56]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomSegmentDataset(cbct_path=paths[i+1], ct_path=paths[i],
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=True, electron=ELECTRON, position="pelvic", segment=N_SEGMENT, g_coord=G_COORD, l_coord=L_COORD)
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "L2_pelvic_ct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "L2_pelvic_ct", patient_id)) 
    iid += len(scans)

31it [00:12,  2.47it/s]
27it [00:11,  2.41it/s]
28it [00:11,  2.46it/s]
27it [00:11,  2.45it/s]
27it [00:09,  2.97it/s]
26it [00:10,  2.40it/s]
30it [00:12,  2.44it/s]
27it [00:11,  2.41it/s]
29it [00:11,  2.50it/s]
26it [00:10,  2.47it/s]
26it [00:10,  2.43it/s]
27it [00:10,  2.47it/s]
26it [00:10,  2.47it/s]
27it [00:11,  2.45it/s]
26it [00:10,  2.45it/s]
30it [00:09,  3.06it/s]
26it [00:10,  2.43it/s]
26it [00:10,  2.40it/s]
26it [00:10,  2.39it/s]
27it [00:10,  2.49it/s]
27it [00:11,  2.45it/s]
26it [00:10,  2.50it/s]
29it [00:11,  2.44it/s]
29it [00:11,  2.51it/s]


In [57]:
testset = DicomsSegmentDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=True, electron=ELECTRON, position="pelvic", segment=N_SEGMENT, g_coord=G_COORD, l_coord=L_COORD)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

656it [02:23,  4.56it/s]


(0.9080922567262882,
 26.706532929001785,
 17.16117582524695,
 0.021164197825383757,
 0.995900253581957,
 0.8553180602493187,
 0.41441777777835365,
 0.4961890243902439,
 0.5007621951219512)

# Abdomen

In [ ]:
test_case_path = 'abdomen/test/*_*'
paths = sorted(glob.glob(test_case_path))

## Abdomen on CBCT

In [ ]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [ ]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "abdomen_test_cbct"))
except FileExistsError:
    pass

In [ ]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomSegmentDataset(cbct_path=paths[i+1], ct_path=paths[i], 
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=False, electron=ELECTRON, position="abdomen", segment=N_SEGMENT, g_coord=G_COORD, l_coord=L_COORD)
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "abdomen_test_cbct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "abdomen_test_cbct", patient_id)) 
    iid += len(scans)

In [ ]:
testset = DicomsSegmentDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=False, electron=ELECTRON, position="abdomen", segment=N_SEGMENT, g_coord=G_COORD, l_coord=L_COORD)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

## Abdomen on CT

In [ ]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [ ]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "abdomen_test_ct"))
except FileExistsError:
    pass

In [ ]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomSegmentDataset(cbct_path=paths[i+1], ct_path=paths[i], 
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=True, electron=ELECTRON, position="abdomen", segment=N_SEGMENT, g_coord=G_COORD, l_coord=L_COORD)
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "abdomen_test_ct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "abdomen_test_ct", patient_id)) 
    iid += len(scans)

In [ ]:
testset = DicomsSegmentDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=True, electron=ELECTRON, position="abdomen", segment=N_SEGMENT, g_coord=G_COORD, l_coord=L_COORD)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

# Chest

In [ ]:
test_case_path = 'chest/test/*_*'
paths = sorted(glob.glob(test_case_path))

## Chest on CBCT

In [ ]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [ ]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "chest_test_cbct"))
except FileExistsError:
    pass

In [ ]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomSegmentDataset(cbct_path=paths[i+1], ct_path=paths[i], 
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=False, electron=ELECTRON, position="chest", segment=N_SEGMENT, g_coord=G_COORD, l_coord=L_COORD)
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "chest_test_cbct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "chest_test_cbct", patient_id)) 
    iid += len(scans)

In [ ]:
testset = DicomsSegmentDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=False, electron=ELECTRON, position="chest", segment=N_SEGMENT, g_coord=G_COORD, l_coord=L_COORD)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

## Chest on CT

In [ ]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [ ]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "chest_test_ct"))
except FileExistsError:
    pass

In [ ]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomSegmentDataset(cbct_path=paths[i+1], ct_path=paths[i], 
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=True, electron=ELECTRON, position="chest",segment=N_SEGMENT, g_coord=G_COORD, l_coord=L_COORD)
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "chest_test_ct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "chest_test_ct", patient_id)) 
    iid += len(scans)

In [ ]:
testset = DicomsSegmentDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=True, electron=ELECTRON, position="chest", segment=N_SEGMENT, g_coord=G_COORD, l_coord=L_COORD)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

# Headneck

In [ ]:
test_case_path = 'headneck/test/*_*'
paths = sorted(glob.glob(test_case_path))

## Headneck on CBCT

In [ ]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [ ]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "hn_test_cbct"))
except FileExistsError:
    pass

In [ ]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomSegmentDataset(cbct_path=paths[i+1], ct_path=paths[i], 
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=False, electron=ELECTRON, position="headneck", segment=N_SEGMENT, g_coord=G_COORD, l_coord=L_COORD)
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "hn_test_cbct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "hn_test_cbct", patient_id)) 
    iid += len(scans)

In [ ]:
testset = DicomsSegmentDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=False, electron=ELECTRON, position="headneck", segment=N_SEGMENT, g_coord=G_COORD, l_coord=L_COORD)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

## Headneck on CT

In [ ]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [ ]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "hn_test_ct"))
except FileExistsError:
    pass

In [ ]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomSegmentDataset(cbct_path=paths[i+1], ct_path=paths[i], 
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=True, electron=ELECTRON, position="headneck", segment=N_SEGMENT, g_coord=G_COORD, l_coord=L_COORD)
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "hn_test_ct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "hn_test_ct", patient_id)) 
    iid += len(scans)

In [ ]:
testset = DicomsSegmentDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=True, electron=ELECTRON, position="headneck", segment=N_SEGMENT, g_coord=G_COORD, l_coord=L_COORD)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)